In [2]:
import matplotlib.pyplot as plt

from turtle import forward
import numpy as np
import pandas as pd
import seaborn as sns
import math
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image

from torch.utils.data import Dataset, DataLoader

In [3]:
import matplotlib.pyplot as plt

def plot_specgram(waveform, sample_rate, title="Spectrogram", file_path=None):
    """
    Draw a matplotlib spectrogram from a `torchaudio.load()` waveform tensor.

    Args
    ----
    waveform : torch.Tensor  # shape (channels, time)
    sample_rate : int
    title : str
    file_path : str | None   # save to PNG if given
    """
    # Convert to NumPy for Matplotlib
    waveform = waveform.numpy()
    channels, frames = waveform.shape

    fig, axes = plt.subplots(channels, 1, figsize=(10, 4 if channels == 1 else 3*channels))

    # axes is ndarray when channels>1; make it iterable in all cases
    axes = axes if isinstance(axes, (list, tuple, np.ndarray)) else [axes]

    for c, ax in enumerate(axes):
        ax.specgram(waveform[c], Fs=sample_rate)
        ax.set_ylabel(f"Channel {c+1}")
    fig.suptitle(title)
    plt.xlabel("Time [s]")
    plt.ylabel("Frequency [Hz]")

    if file_path:
        plt.savefig(file_path, dpi=150, bbox_inches="tight")
    else:
        plt.show()
    plt.close(fig)


from pathlib import Path
import torchaudio
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms  

class AudioFolder(Dataset):
    def __init__(self, root, transform=None, target_transform=None,
                 extensions=(".wav", ".flac", ".mp3")):
        self.root = Path(root)
        self.transform = transform
        self.target_transform = target_transform
        self.extensions = extensions

        # crawl sub-dirs and record (path, class_idx)
        self.classes = sorted({p.parent.name for p in self.root.rglob("*") if p.suffix in extensions})
        self.class_to_idx = {c: i for i, c in enumerate(self.classes)}
        self.samples = [(p, self.class_to_idx[p.parent.name])
                        for p in self.root.rglob("*") if p.suffix in extensions]

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        waveform, sr = torchaudio.load(path)

        if self.transform:
            waveform = self.transform(waveform)
        if self.target_transform:
            label = self.target_transform(label)

        return waveform, sr, label


In [19]:
import torchaudio, random

data_path = '../data/hb/'
unl = 'unlabelled'

unlab = Path(data_path,unl)

random_unlabel = random.choice(list(unlab.glob('*')))


waveform, sr = torchaudio.load(random_unlabel.resolve().as_uri(), backend="sox_io")
plot_specgram(waveform, sr, file_path="random_unlabel.png")  

ValueError: ("Unsupported backend 'sox_io' specified; ", 'please select one of [] instead.')

In [1]:
!ls ../data/hb/unlabelled/unlabelled__0004.wav


../data/hb/unlabelled/unlabelled__0004.wav


In [2]:
import torchaudio
print(torchaudio.list_audio_backends())

[]


In [21]:
import torch
print(torch.version.cuda)        # CUDA version PyTorch was built with
print(torch.backends.cudnn.version())  # cuDNN version (if available)
print(torch.cuda.is_available())      # whether CUDA runtime is accessible

12.4
90100
True
